# 📊 Notebook 4 : Évaluation Approfondie des Modèles

## Objectif : Analyser les performances, les erreurs et préparer le déploiement

### Ce notebook couvre :
1. Chargement des modèles entraînés
2. Analyse détaillée des performances
3. Analyse des erreurs de prédiction
4. Courbes d'apprentissage et validation croisée
5. Analyse SHAP pour l'interprétabilité
6. Recommandations sectorielles
7. Rapport final et préparation au déploiement
8. Export des résultats pour l'API

## 1. 📦 Import des bibliothèques

In [1]:
# Bibliothèques de base
import pandas as pd
import numpy as np
from pathlib import Path
import json
import pickle
import warnings
warnings.filterwarnings('ignore')

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Machine Learning
from sklearn.model_selection import cross_val_score, learning_curve
from sklearn.metrics import (mean_squared_error, mean_absolute_error, r2_score,
                            accuracy_score, classification_report, confusion_matrix,
                            precision_recall_curve, roc_curve, auc,
                            precision_score, recall_score, f1_score)

# XGBoost
import xgboost as xgb

# Configuration
plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

print("✅ Bibliothèques importées avec succès!")

✅ Bibliothèques importées avec succès!


## 2. 📂 Chargement des modèles et données

In [2]:
# Chemins
DATA_PATH = Path("../data/processed")
MODELS_PATH = Path("../models")

# Charger les données
df = pd.read_csv(DATA_PATH / "dataset_ml_complete.csv")

# Charger les modèles
with open(MODELS_PATH / "xgb_regression_substitution.pkl", 'rb') as f:
    model_reg = pickle.load(f)

with open(MODELS_PATH / "xgb_classification_opportunity.pkl", 'rb') as f:
    model_clf = pickle.load(f)

with open(MODELS_PATH / "scaler.pkl", 'rb') as f:
    scaler = pickle.load(f)

# Charger les métadonnées
with open(MODELS_PATH / "model_metadata.json", 'r', encoding='utf-8') as f:
    model_metadata = json.load(f)

# Charger l'analyse sectorielle
sector_opportunities = pd.read_csv(MODELS_PATH / "sector_opportunities.csv", index_col=0)

print("✅ Modèles et données chargés avec succès!")
print(f"\n📊 Dataset: {df.shape[0]} lignes × {df.shape[1]} colonnes")
print(f"📅 Date d'entraînement: {model_metadata['date_training']}")
print(f"\n📈 Modèle de régression - R²: {model_metadata['regression_model']['metrics']['r2']:.4f}")
print(f"📈 Modèle de classification - Accuracy: {model_metadata['classification_model']['metrics']['accuracy']:.4f}")

✅ Modèles et données chargés avec succès!

📊 Dataset: 210 lignes × 52 colonnes
📅 Date d'entraînement: 2025-12-19 17:35:10

📈 Modèle de régression - R²: 0.8320
📈 Modèle de classification - Accuracy: 0.9841


In [3]:
# Recréer les variables cibles et préparer les données
df_eval = df.copy()

# Recréer le score de substitution
df_eval['substitution_score'] = np.where(
    df_eval['imports_fcfa'] > 0,
    df_eval['production_fcfa'] / df_eval['imports_fcfa'],
    df_eval['production_fcfa']
)
df_eval['substitution_score_normalized'] = (
    (df_eval['substitution_score'] - df_eval['substitution_score'].min()) / 
    (df_eval['substitution_score'].max() - df_eval['substitution_score'].min()) * 100
)

# Recréer les classes d'opportunité
def classify_opportunity(row):
    score = 0
    ratio = row['production_fcfa'] / row['imports_fcfa'] if row['imports_fcfa'] > 0 else row['production_fcfa']
    demande = row['production_fcfa'] + row['imports_fcfa'] - row['exports_fcfa']
    autosuff = (row['production_fcfa'] / demande * 100) if demande > 0 else 0
    
    if ratio > 5: score += 3
    elif ratio > 1: score += 2
    elif ratio > 0.5: score += 1
    
    if autosuff > 80: score += 3
    elif autosuff > 50: score += 2
    elif autosuff > 30: score += 1
    
    if row['production_fcfa_growth'] > 10: score += 2
    elif row['production_fcfa_growth'] > 0: score += 1
    
    if score >= 6: return 2
    elif score >= 3: return 1
    else: return 0

df_eval['opportunity_class'] = df_eval.apply(classify_opportunity, axis=1)
opportunity_labels = {0: 'Faible', 1: 'Moyenne', 2: 'Haute'}
df_eval['opportunity_label'] = df_eval['opportunity_class'].map(opportunity_labels)

# Features
feature_cols = model_metadata['feature_columns']
train_years = model_metadata['train_years']
test_years = model_metadata['test_years']

# Split train/test
train_mask = df_eval['year'].isin(train_years)
test_mask = df_eval['year'].isin(test_years)

X_train = df_eval.loc[train_mask, feature_cols].values
X_test = df_eval.loc[test_mask, feature_cols].values
y_train_reg = df_eval.loc[train_mask, 'substitution_score_normalized'].values
y_test_reg = df_eval.loc[test_mask, 'substitution_score_normalized'].values
y_train_clf = df_eval.loc[train_mask, 'opportunity_class'].values
y_test_clf = df_eval.loc[test_mask, 'opportunity_class'].values

# Normalisation
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("✅ Données préparées pour l'évaluation")
print(f"   Train: {len(X_train)} échantillons")
print(f"   Test: {len(X_test)} échantillons")

✅ Données préparées pour l'évaluation
   Train: 147 échantillons
   Test: 63 échantillons


## 3. 📊 Analyse détaillée des performances

In [4]:
# 3.1 Performance du modèle de régression
y_pred_reg = model_reg.predict(X_test_scaled)
y_pred_train_reg = model_reg.predict(X_train_scaled)

# Métriques détaillées
metrics_reg = {
    'Train': {
        'RMSE': np.sqrt(mean_squared_error(y_train_reg, y_pred_train_reg)),
        'MAE': mean_absolute_error(y_train_reg, y_pred_train_reg),
        'R²': r2_score(y_train_reg, y_pred_train_reg),
        'MAPE': np.mean(np.abs((y_train_reg - y_pred_train_reg) / (y_train_reg + 1e-10))) * 100
    },
    'Test': {
        'RMSE': np.sqrt(mean_squared_error(y_test_reg, y_pred_reg)),
        'MAE': mean_absolute_error(y_test_reg, y_pred_reg),
        'R²': r2_score(y_test_reg, y_pred_reg),
        'MAPE': np.mean(np.abs((y_test_reg - y_pred_reg) / (y_test_reg + 1e-10))) * 100
    }
}

print("📊 PERFORMANCE DU MODÈLE DE RÉGRESSION")
print("="*60)
print(f"\n{'Métrique':<15} {'Train':>15} {'Test':>15} {'Écart':>15}")
print("-"*60)
for metric in ['RMSE', 'MAE', 'R²', 'MAPE']:
    train_val = metrics_reg['Train'][metric]
    test_val = metrics_reg['Test'][metric]
    ecart = abs(train_val - test_val)
    print(f"{metric:<15} {train_val:>15.4f} {test_val:>15.4f} {ecart:>15.4f}")

# Analyse de l'overfitting
overfit_ratio = metrics_reg['Train']['R²'] / metrics_reg['Test']['R²']
print(f"\n📈 Ratio Train/Test R²: {overfit_ratio:.2f}")
if overfit_ratio > 1.2:
    print("   ⚠️ Légère tendance à l'overfitting détectée")
else:
    print("   ✅ Pas d'overfitting significatif")

📊 PERFORMANCE DU MODÈLE DE RÉGRESSION

Métrique                  Train            Test           Écart
------------------------------------------------------------
RMSE                     6.8975          3.6252          3.2723
MAE                      2.3108          1.8506          0.4602
R²                       0.7639          0.8320          0.0681
MAPE            22166740059.2157 17240798556.3375 4925941502.8782

📈 Ratio Train/Test R²: 0.92
   ✅ Pas d'overfitting significatif


In [5]:
# 3.2 Visualisation des performances de régression
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Prédictions vs Réalité (Test)',
        'Résidus vs Prédictions',
        'Distribution des Erreurs',
        'Q-Q Plot des Résidus'
    )
)

# 1. Scatter plot prédictions vs réalité
fig.add_trace(
    go.Scatter(x=y_test_reg, y=y_pred_reg, mode='markers',
               marker=dict(color='steelblue', size=8, opacity=0.7),
               name='Prédictions'),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=[0, max(y_test_reg)], y=[0, max(y_test_reg)], mode='lines',
               line=dict(color='red', dash='dash'), name='Parfait'),
    row=1, col=1
)

# 2. Résidus vs Prédictions
residuals = y_test_reg - y_pred_reg
fig.add_trace(
    go.Scatter(x=y_pred_reg, y=residuals, mode='markers',
               marker=dict(color='#e74c3c', size=8, opacity=0.7),
               name='Résidus'),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(x=[min(y_pred_reg), max(y_pred_reg)], y=[0, 0], mode='lines',
               line=dict(color='black', dash='dash')),
    row=1, col=2
)

# 3. Distribution des erreurs
fig.add_trace(
    go.Histogram(x=residuals, nbinsx=20, marker_color='steelblue', name='Erreurs'),
    row=2, col=1
)

# 4. Q-Q Plot approximatif
sorted_residuals = np.sort(residuals)
theoretical_quantiles = np.linspace(0.01, 0.99, len(sorted_residuals))
from scipy import stats
theoretical_values = stats.norm.ppf(theoretical_quantiles, loc=np.mean(residuals), scale=np.std(residuals))
fig.add_trace(
    go.Scatter(x=theoretical_values, y=sorted_residuals, mode='markers',
               marker=dict(color='#27ae60', size=6), name='Q-Q'),
    row=2, col=2
)
fig.add_trace(
    go.Scatter(x=[min(theoretical_values), max(theoretical_values)], 
               y=[min(theoretical_values), max(theoretical_values)],
               mode='lines', line=dict(color='red', dash='dash')),
    row=2, col=2
)

fig.update_layout(
    title_text="📊 Analyse des Performances - Modèle de Régression",
    height=700, showlegend=False
)
fig.show()

In [6]:
# 3.3 Performance du modèle de classification
y_pred_clf = model_clf.predict(X_test_scaled)
y_pred_train_clf = model_clf.predict(X_train_scaled)

# Métriques par classe
print("📊 PERFORMANCE DU MODÈLE DE CLASSIFICATION")
print("="*60)

print("\n📋 Rapport de classification (Test):")
print(classification_report(y_test_clf, y_pred_clf, 
                           target_names=['Faible', 'Moyenne', 'Haute']))

# Matrice de confusion
cm = confusion_matrix(y_test_clf, y_pred_clf)

fig = go.Figure(data=go.Heatmap(
    z=cm,
    x=['Faible', 'Moyenne', 'Haute'],
    y=['Faible', 'Moyenne', 'Haute'],
    colorscale='Blues',
    text=cm,
    texttemplate="%{text}",
    textfont={"size": 20}
))

fig.update_layout(
    title="🎯 Matrice de Confusion - Classification des Opportunités",
    xaxis_title="Classe Prédite",
    yaxis_title="Classe Réelle",
    height=400
)
fig.show()

# Métriques globales
print(f"\n📈 Métriques globales:")
print(f"   Accuracy: {accuracy_score(y_test_clf, y_pred_clf):.4f}")
print(f"   Precision (macro): {precision_score(y_test_clf, y_pred_clf, average='macro'):.4f}")
print(f"   Recall (macro): {recall_score(y_test_clf, y_pred_clf, average='macro'):.4f}")
print(f"   F1-Score (macro): {f1_score(y_test_clf, y_pred_clf, average='macro'):.4f}")

📊 PERFORMANCE DU MODÈLE DE CLASSIFICATION

📋 Rapport de classification (Test):
              precision    recall  f1-score   support

      Faible       1.00      1.00      1.00         2
     Moyenne       0.97      1.00      0.98        29
       Haute       1.00      0.97      0.98        32

    accuracy                           0.98        63
   macro avg       0.99      0.99      0.99        63
weighted avg       0.98      0.98      0.98        63




📈 Métriques globales:
   Accuracy: 0.9841
   Precision (macro): 0.9889
   Recall (macro): 0.9896
   F1-Score (macro): 0.9891


## 4. 🔍 Analyse des erreurs de prédiction

In [7]:
# 4.1 Identifier les cas avec les plus grandes erreurs
df_test = df_eval[test_mask].copy()
df_test['predicted_score'] = y_pred_reg
df_test['actual_score'] = y_test_reg
df_test['error'] = df_test['actual_score'] - df_test['predicted_score']
df_test['abs_error'] = np.abs(df_test['error'])
df_test['predicted_class'] = y_pred_clf
df_test['actual_class'] = y_test_clf
df_test['class_correct'] = df_test['predicted_class'] == df_test['actual_class']

# Top erreurs de régression
print("🔍 ANALYSE DES ERREURS DE PRÉDICTION")
print("="*60)

print("\n📊 Top 10 des plus grandes erreurs (Régression):")
print("-"*60)
top_errors = df_test.nlargest(10, 'abs_error')[['LIBELLES', 'year', 'actual_score', 'predicted_score', 'error']]
for i, (idx, row) in enumerate(top_errors.iterrows(), 1):
    print(f"{i:2d}. {row['LIBELLES'][:40]}... ({row['year']})")
    print(f"    Réel: {row['actual_score']:.2f} | Prédit: {row['predicted_score']:.2f} | Erreur: {row['error']:+.2f}")

# Erreurs de classification
misclassified = df_test[~df_test['class_correct']]
print(f"\n📊 Erreurs de classification: {len(misclassified)}/{len(df_test)} ({len(misclassified)/len(df_test)*100:.1f}%)")
if len(misclassified) > 0:
    print("\nCas mal classifiés:")
    for idx, row in misclassified.iterrows():
        print(f"   • {row['LIBELLES'][:40]}... ({row['year']})")
        print(f"     Réel: {opportunity_labels[row['actual_class']]} → Prédit: {opportunity_labels[row['predicted_class']]}")

🔍 ANALYSE DES ERREURS DE PRÉDICTION

📊 Top 10 des plus grandes erreurs (Régression):
------------------------------------------------------------
 1. Perles fines ou de culture, pierres gemm... (2021)
    Réel: 52.96 | Prédit: 31.05 | Erreur: +21.91
 2. Matières textiles et ouvrages en ces mat... (2022)
    Réel: 0.09 | Prédit: 10.23 | Erreur: -10.14
 3. Matières textiles et ouvrages en ces mat... (2021)
    Réel: 0.09 | Prédit: 10.23 | Erreur: -10.14
 4. Perles fines ou de culture, pierres gemm... (2023)
    Réel: 34.62 | Prédit: 28.23 | Erreur: +6.40
 5. Perles fines ou de culture, pierres gemm... (2022)
    Réel: 34.11 | Prédit: 28.23 | Erreur: +5.89
 6. Objets d'art, de collection ou d'antiqui... (2021)
    Réel: 0.00 | Prédit: 1.09 | Erreur: -1.09
 7. Armes, munitions et leurs parties et acc... (2021)
    Réel: 0.01 | Prédit: 1.09 | Erreur: -1.07
 8. Armes, munitions et leurs parties et acc... (2023)
    Réel: 0.01 | Prédit: 1.09 | Erreur: -1.07
 9. Ouvrages en pierres, plâtres, c

In [8]:
# 4.2 Analyse des erreurs par secteur et par année
error_by_sector = df_test.groupby('LIBELLES')['abs_error'].mean().sort_values(ascending=False)
error_by_year = df_test.groupby('year')['abs_error'].mean()

fig = make_subplots(rows=1, cols=2,
                    subplot_titles=('Erreur moyenne par secteur (Top 10)', 
                                   'Erreur moyenne par année'))

# Erreur par secteur
fig.add_trace(
    go.Bar(x=error_by_sector.head(10).values,
           y=[s[:30]+'...' for s in error_by_sector.head(10).index],
           orientation='h',
           marker_color='#e74c3c'),
    row=1, col=1
)

# Erreur par année
fig.add_trace(
    go.Bar(x=error_by_year.index.astype(str),
           y=error_by_year.values,
           marker_color='steelblue'),
    row=1, col=2
)

fig.update_layout(
    title_text="🔍 Distribution des Erreurs",
    height=400,
    showlegend=False
)
fig.show()

print("\n📊 Statistiques des erreurs:")
print(f"   Erreur moyenne: {df_test['abs_error'].mean():.4f}")
print(f"   Erreur médiane: {df_test['abs_error'].median():.4f}")
print(f"   Écart-type: {df_test['abs_error'].std():.4f}")
print(f"   Erreur max: {df_test['abs_error'].max():.4f}")


📊 Statistiques des erreurs:
   Erreur moyenne: 1.8506
   Erreur médiane: 1.0740
   Écart-type: 3.1423
   Erreur max: 21.9113


## 5. 📈 Validation croisée et stabilité du modèle

In [9]:
# 5.1 Validation croisée sur l'ensemble des données
from sklearn.model_selection import cross_val_score, KFold

# Préparer toutes les données
X_all = df_eval[feature_cols].values
y_all_reg = df_eval['substitution_score_normalized'].values
y_all_clf = df_eval['opportunity_class'].values
X_all_scaled = scaler.fit_transform(X_all)

# Cross-validation pour la régression
cv = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores_reg = cross_val_score(model_reg, X_all_scaled, y_all_reg, cv=cv, scoring='r2')
cv_scores_rmse = cross_val_score(model_reg, X_all_scaled, y_all_reg, cv=cv, 
                                  scoring='neg_root_mean_squared_error')

print("📈 VALIDATION CROISÉE (5-Fold)")
print("="*60)
print(f"\n🔹 Modèle de Régression:")
print(f"   R² scores: {cv_scores_reg}")
print(f"   R² moyen: {cv_scores_reg.mean():.4f} (+/- {cv_scores_reg.std()*2:.4f})")
print(f"   RMSE moyen: {-cv_scores_rmse.mean():.4f} (+/- {cv_scores_rmse.std()*2:.4f})")

# Cross-validation pour la classification
cv_scores_clf = cross_val_score(model_clf, X_all_scaled, y_all_clf, cv=cv, scoring='accuracy')
cv_scores_f1 = cross_val_score(model_clf, X_all_scaled, y_all_clf, cv=cv, scoring='f1_macro')

print(f"\n🔹 Modèle de Classification:")
print(f"   Accuracy scores: {cv_scores_clf}")
print(f"   Accuracy moyenne: {cv_scores_clf.mean():.4f} (+/- {cv_scores_clf.std()*2:.4f})")
print(f"   F1-macro moyen: {cv_scores_f1.mean():.4f} (+/- {cv_scores_f1.std()*2:.4f})")

# Visualisation
fig = make_subplots(rows=1, cols=2,
                    subplot_titles=('Scores CV - Régression (R²)', 
                                   'Scores CV - Classification (Accuracy)'))

fig.add_trace(
    go.Bar(x=[f'Fold {i+1}' for i in range(5)], y=cv_scores_reg,
           marker_color='steelblue'),
    row=1, col=1
)
fig.add_hline(y=cv_scores_reg.mean(), line_dash="dash", line_color="red", row=1, col=1)

fig.add_trace(
    go.Bar(x=[f'Fold {i+1}' for i in range(5)], y=cv_scores_clf,
           marker_color='#27ae60'),
    row=1, col=2
)
fig.add_hline(y=cv_scores_clf.mean(), line_dash="dash", line_color="red", row=1, col=2)

fig.update_layout(title_text="📈 Stabilité des Modèles - Validation Croisée", 
                  height=400, showlegend=False)
fig.show()

📈 VALIDATION CROISÉE (5-Fold)

🔹 Modèle de Régression:
   R² scores: [0.66171082 0.51083654 0.80309243 0.82184817 0.7256551 ]
   R² moyen: 0.7046 (+/- 0.2250)
   RMSE moyen: 6.8897 (+/- 5.4413)

🔹 Modèle de Classification:
   Accuracy scores: [0.97619048 0.92857143 0.97619048 1.         1.        ]
   Accuracy moyenne: 0.9762 (+/- 0.0522)
   F1-macro moyen: 0.9694 (+/- 0.0872)


## 6. 🎯 Importance des Features et Interprétabilité

In [10]:
# 6.1 Importance des features - Comparaison des deux modèles
importance_reg = pd.DataFrame({
    'feature': feature_cols,
    'importance_reg': model_reg.feature_importances_
})

importance_clf = pd.DataFrame({
    'feature': feature_cols,
    'importance_clf': model_clf.feature_importances_
})

# Fusionner
importance_combined = importance_reg.merge(importance_clf, on='feature')
importance_combined['importance_avg'] = (importance_combined['importance_reg'] + 
                                          importance_combined['importance_clf']) / 2
importance_combined = importance_combined.sort_values('importance_avg', ascending=False)

# Top 15 features
top_15 = importance_combined.head(15)

fig = go.Figure()

fig.add_trace(go.Bar(
    name='Régression',
    y=top_15['feature'],
    x=top_15['importance_reg'],
    orientation='h',
    marker_color='steelblue'
))

fig.add_trace(go.Bar(
    name='Classification',
    y=top_15['feature'],
    x=top_15['importance_clf'],
    orientation='h',
    marker_color='#27ae60'
))

fig.update_layout(
    title="🎯 Top 15 Features les Plus Importantes",
    barmode='group',
    height=500,
    xaxis_title="Importance",
    yaxis_title="Feature",
    yaxis={'categoryorder': 'total ascending'}
)
fig.show()

print("📊 Top 10 Features (moyenne des deux modèles):")
for i, row in importance_combined.head(10).iterrows():
    print(f"   {row['importance_avg']:.4f} - {row['feature']}")

📊 Top 10 Features (moyenne des deux modèles):
   0.2191 - prix_unitaire_production
   0.1697 - consommation_fcfa
   0.1668 - production_fcfa_growth
   0.1273 - exports_fcfa
   0.0870 - taux_couverture
   0.0766 - balance_commerciale_fcfa
   0.0556 - ratio_prod_conso_fcfa
   0.0254 - imports_fcfa_growth
   0.0213 - intensite_export
   0.0101 - production_tonnes_growth


In [11]:
# 6.2 Interprétation économique des features importantes
print("📊 INTERPRÉTATION ÉCONOMIQUE DES FEATURES CLÉS")
print("="*70)

interpretations = {
    'prix_unitaire_production': """
    💰 Prix Unitaire de Production (FCFA/Tonne)
    → Indicateur de valeur ajoutée du secteur
    → Plus il est élevé, plus le secteur transforme les matières premières
    → Secteurs à fort prix unitaire = opportunités de substitution à haute valeur""",
    
    'production_fcfa_growth': """
    📈 Croissance de la Production
    → Dynamisme du secteur sur l'année
    → Secteurs en croissance = capacité d'expansion pour remplacer les imports
    → Signal positif pour les investissements""",
    
    'consommation_fcfa': """
    🏠 Consommation Intérieure
    → Demande locale pour les produits du secteur
    → Forte consommation = marché potentiel pour la production locale
    → Opportunité si production < consommation""",
    
    'exports_fcfa': """
    🌍 Exportations
    → Compétitivité internationale du secteur
    → Secteurs exportateurs = capacité à être compétitifs
    → Potentiel de réorienter les exports vers le marché local""",
    
    'taux_couverture': """
    ⚖️ Taux de Couverture (Exports/Imports)
    → Équilibre commercial du secteur
    → > 100% = secteur excédentaire (fort potentiel)
    → < 100% = secteur déficitaire (besoin de substitution)"""
}

for feature in importance_combined.head(5)['feature'].values:
    if feature in interpretations:
        print(interpretations[feature])
        print("-"*70)

📊 INTERPRÉTATION ÉCONOMIQUE DES FEATURES CLÉS

    💰 Prix Unitaire de Production (FCFA/Tonne)
    → Indicateur de valeur ajoutée du secteur
    → Plus il est élevé, plus le secteur transforme les matières premières
    → Secteurs à fort prix unitaire = opportunités de substitution à haute valeur
----------------------------------------------------------------------

    🏠 Consommation Intérieure
    → Demande locale pour les produits du secteur
    → Forte consommation = marché potentiel pour la production locale
    → Opportunité si production < consommation
----------------------------------------------------------------------

    📈 Croissance de la Production
    → Dynamisme du secteur sur l'année
    → Secteurs en croissance = capacité d'expansion pour remplacer les imports
    → Signal positif pour les investissements
----------------------------------------------------------------------

    🌍 Exportations
    → Compétitivité internationale du secteur
    → Secteurs exportateurs

## 7. 🏆 Recommandations Sectorielles

In [12]:
# 7.1 Analyse approfondie par secteur
# Calculer les statistiques moyennes récentes (2021-2023) par secteur
recent_data = df_eval[df_eval['year'].isin([2021, 2022, 2023])].copy()

sector_analysis = recent_data.groupby('LIBELLES').agg({
    'production_fcfa': 'mean',
    'imports_fcfa': 'mean',
    'exports_fcfa': 'mean',
    'consommation_fcfa': 'mean',
    'production_fcfa_growth': 'mean',
    'substitution_score_normalized': 'mean'
}).round(2)

# Calculer des métriques supplémentaires
sector_analysis['balance_commerciale'] = sector_analysis['exports_fcfa'] - sector_analysis['imports_fcfa']
sector_analysis['ratio_prod_import'] = sector_analysis['production_fcfa'] / sector_analysis['imports_fcfa'].replace(0, 1)
sector_analysis['potentiel_substitution'] = sector_analysis['imports_fcfa'] * (1 - sector_analysis['ratio_prod_import'].clip(0, 1))

# Classifier les secteurs
def classify_sector(row):
    if row['ratio_prod_import'] > 5 and row['production_fcfa_growth'] > 0:
        return '🟢 Fort potentiel - Secteur dominant'
    elif row['ratio_prod_import'] > 1:
        return '🟡 Potentiel modéré - Équilibré'
    elif row['ratio_prod_import'] > 0.5:
        return '🟠 À développer - Dépendant'
    else:
        return '🔴 Prioritaire - Très dépendant'

sector_analysis['classification'] = sector_analysis.apply(classify_sector, axis=1)
sector_analysis = sector_analysis.sort_values('substitution_score_normalized', ascending=False)

print("🏆 RECOMMANDATIONS SECTORIELLES")
print("="*80)
print("\n📊 Classement des secteurs par potentiel de substitution aux importations:\n")

for i, (sector, row) in enumerate(sector_analysis.head(10).iterrows(), 1):
    print(f"{i:2d}. {sector[:50]}...")
    print(f"    {row['classification']}")
    print(f"    Production: {row['production_fcfa']:.1f} Mds | Imports: {row['imports_fcfa']:.1f} Mds")
    print(f"    Ratio Prod/Import: {row['ratio_prod_import']:.2f} | Croissance: {row['production_fcfa_growth']:+.1f}%")
    print()

🏆 RECOMMANDATIONS SECTORIELLES

📊 Classement des secteurs par potentiel de substitution aux importations:

 1. Perles fines ou de culture, pierres gemmes ou simi...
    🟢 Fort potentiel - Secteur dominant
    Production: 2138.1 Mds | Imports: 0.8 Mds
    Ratio Prod/Import: 2672.62 | Croissance: +1.4%

 2. Matières textiles et ouvrages en ces matières...
    🟢 Fort potentiel - Secteur dominant
    Production: 286.2 Mds | Imports: 48.4 Mds
    Ratio Prod/Import: 5.91 | Croissance: +7.3%

 3. Produit du règne végétal...
    🟡 Potentiel modéré - Équilibré
    Production: 343.1 Mds | Imports: 165.4 Mds
    Ratio Prod/Import: 2.07 | Croissance: +14.0%

 4. Graisses et huiles animales ou végétales...
    🟡 Potentiel modéré - Équilibré
    Production: 36.9 Mds | Imports: 18.6 Mds
    Ratio Prod/Import: 1.98 | Croissance: +9.3%

 5. Objets d'art, de collection ou d'antiquité...
    🟡 Potentiel modéré - Équilibré
    Production: 0.2 Mds | Imports: 0.1 Mds
    Ratio Prod/Import: 2.86 | Croissance

In [13]:
# 7.2 Visualisation des recommandations
fig = px.scatter(
    sector_analysis.reset_index(),
    x='imports_fcfa',
    y='production_fcfa',
    size='substitution_score_normalized',
    color='production_fcfa_growth',
    hover_name='LIBELLES',
    color_continuous_scale='RdYlGn',
    title='🎯 Cartographie des Opportunités de Substitution',
    labels={
        'imports_fcfa': 'Importations (Mds FCFA)',
        'production_fcfa': 'Production (Mds FCFA)',
        'production_fcfa_growth': 'Croissance (%)'
    }
)

# Ajouter la ligne de parité
max_val = max(sector_analysis['imports_fcfa'].max(), sector_analysis['production_fcfa'].max())
fig.add_trace(go.Scatter(
    x=[0, max_val], y=[0, max_val],
    mode='lines',
    line=dict(color='gray', dash='dash'),
    name='Parité Prod=Import'
))

fig.add_annotation(
    x=max_val*0.7, y=max_val*0.3,
    text="Zone de dépendance<br>aux importations",
    showarrow=False,
    font=dict(size=10, color='red')
)

fig.add_annotation(
    x=max_val*0.3, y=max_val*0.7,
    text="Zone de<br>substitution réussie",
    showarrow=False,
    font=dict(size=10, color='green')
)

fig.update_layout(height=600)
fig.show()

In [14]:
# 7.3 Secteurs prioritaires pour l'action
print("🎯 SECTEURS PRIORITAIRES POUR L'ACTION")
print("="*80)

# Identifier les secteurs à fort potentiel de substitution non exploité
high_import_sectors = sector_analysis[sector_analysis['imports_fcfa'] > 50].copy()
high_import_sectors['gap'] = high_import_sectors['imports_fcfa'] - high_import_sectors['production_fcfa']
high_import_sectors = high_import_sectors[high_import_sectors['gap'] > 0].sort_values('gap', ascending=False)

print("\n🔴 SECTEURS AVEC FORT DÉFICIT (Imports >> Production):")
print("-"*80)
for i, (sector, row) in enumerate(high_import_sectors.head(5).iterrows(), 1):
    print(f"\n{i}. {sector}")
    print(f"   💰 Gap à combler: {row['gap']:.1f} Mds FCFA")
    print(f"   📊 Imports: {row['imports_fcfa']:.1f} Mds | Production: {row['production_fcfa']:.1f} Mds")
    print(f"   📈 Croissance actuelle: {row['production_fcfa_growth']:+.1f}%")
    
    # Recommandation
    if row['production_fcfa_growth'] > 5:
        print(f"   ✅ RECOMMANDATION: Accélérer la croissance existante")
    elif row['production_fcfa_growth'] > 0:
        print(f"   ⚠️ RECOMMANDATION: Stimuler davantage le secteur")
    else:
        print(f"   🚨 RECOMMANDATION: Intervention urgente nécessaire")

# Secteurs champions
print("\n\n🟢 SECTEURS CHAMPIONS (Production >> Imports):")
print("-"*80)
champions = sector_analysis[sector_analysis['ratio_prod_import'] > 2].head(5)
for i, (sector, row) in enumerate(champions.iterrows(), 1):
    print(f"\n{i}. {sector}")
    print(f"   🏆 Ratio Prod/Import: {row['ratio_prod_import']:.1f}x")
    print(f"   💡 Potentiel d'exportation additionnel ou de diversification")

🎯 SECTEURS PRIORITAIRES POUR L'ACTION

🔴 SECTEURS AVEC FORT DÉFICIT (Imports >> Production):
--------------------------------------------------------------------------------


🟢 SECTEURS CHAMPIONS (Production >> Imports):
--------------------------------------------------------------------------------

1. Perles fines ou de culture, pierres gemmes ou similaires, métaux précieux, plaques ou doubles de métaux précieux et ouvrages en ces matières; bijouterie de fantaisie; monnaies
   🏆 Ratio Prod/Import: 2672.6x
   💡 Potentiel d'exportation additionnel ou de diversification

2. Matières textiles et ouvrages en ces matières
   🏆 Ratio Prod/Import: 5.9x
   💡 Potentiel d'exportation additionnel ou de diversification

3. Produit du règne végétal
   🏆 Ratio Prod/Import: 2.1x
   💡 Potentiel d'exportation additionnel ou de diversification

4. Objets d'art, de collection ou d'antiquité
   🏆 Ratio Prod/Import: 2.9x
   💡 Potentiel d'exportation additionnel ou de diversification


## 8. 💾 Export des résultats pour l'API et le déploiement

In [15]:
# 8.1 Préparer les données pour l'API
api_data = {
    'model_info': {
        'version': '1.0',
        'date_training': model_metadata['date_training'],
        'regression_r2': model_metadata['regression_model']['metrics']['r2'],
        'classification_accuracy': model_metadata['classification_model']['metrics']['accuracy']
    },
    'feature_columns': feature_cols,
    'opportunity_classes': opportunity_labels,
    'sectors': df_eval['LIBELLES'].unique().tolist()
}

# Sauvegarder les données API
with open(MODELS_PATH / "api_config.json", 'w', encoding='utf-8') as f:
    json.dump(api_data, f, ensure_ascii=False, indent=2)

print("✅ Configuration API sauvegardée: api_config.json")

✅ Configuration API sauvegardée: api_config.json


In [16]:
# 8.2 Créer le rapport final des recommandations
recommendations_report = []

for sector, row in sector_analysis.iterrows():
    rec = {
        'secteur': sector,
        'production_mds_fcfa': round(row['production_fcfa'], 2),
        'imports_mds_fcfa': round(row['imports_fcfa'], 2),
        'exports_mds_fcfa': round(row['exports_fcfa'], 2),
        'balance_commerciale': round(row['balance_commerciale'], 2),
        'ratio_production_imports': round(row['ratio_prod_import'], 2),
        'croissance_production_pct': round(row['production_fcfa_growth'], 2),
        'score_substitution': round(row['substitution_score_normalized'], 2),
        'classification': row['classification'],
        'potentiel_substitution_mds': round(max(0, row['imports_fcfa'] - row['production_fcfa']), 2)
    }
    recommendations_report.append(rec)

# Sauvegarder en JSON
with open(MODELS_PATH / "recommendations_report.json", 'w', encoding='utf-8') as f:
    json.dump(recommendations_report, f, ensure_ascii=False, indent=2)

# Sauvegarder en CSV
recommendations_df = pd.DataFrame(recommendations_report)
recommendations_df.to_csv(MODELS_PATH / "recommendations_report.csv", index=False, encoding='utf-8')

print("✅ Rapport de recommandations sauvegardé:")
print("   📁 recommendations_report.json")
print("   📁 recommendations_report.csv")

# Aperçu
recommendations_df.head(10)

✅ Rapport de recommandations sauvegardé:
   📁 recommendations_report.json
   📁 recommendations_report.csv


,secteur,production_mds_fcfa,imports_mds_fcfa,exports_mds_fcfa,balance_commerciale,ratio_production_imports,croissance_production_pct,score_substitution,classification,potentiel_substitution_mds
0,"Perles fines ou de culture, pierres gemmes ou ...",2138.100,0.800,2137.300,2136.500,2672.620,1.350,40.570,🟢 Fort potentiel - Secteur dominant,0
1,Matières textiles et ouvrages en ces matières,286.230,48.400,237.830,189.430,5.910,7.310,0.080,🟢 Fort potentiel - Secteur dominant,0
2,Produit du règne végétal,343.130,165.400,177.730,12.330,2.070,13.980,0.030,🟡 Potentiel modéré - Équilibré,0
3,Graisses et huiles animales ou végétales,36.900,18.600,18.300,-0.300,1.980,9.260,0.030,🟡 Potentiel modéré - Équilibré,0
4,"Objets d'art, de collection ou d'antiquité",0.200,0.070,0.130,0.060,2.860,0.000,0.020,🟡 Potentiel modéré - Équilibré,0
5,"Chaussures, coiffures, parapluie, cannes, etc.",7.700,7.600,0.100,-7.500,1.010,17.540,0.010,🟡 Potentiel modéré - Équilibré,0
6,"Bois, charbons de bois et ouvrages en bois",8.100,7.830,0.270,-7.560,1.030,27.720,0.010,🟡 Potentiel modéré - Équilibré,0
7,"Armes, munitions et leurs parties et accessoires",1.400,1.370,0.030,-1.340,1.020,0.790,0.010,🟡 Potentiel modéré - Équilibré,0
8,Animaux vivants et produits du règne animal,38.770,36.800,1.970,-34.830,1.050,20.170,0.010,🟡 Potentiel modéré - Équilibré,0
9,Matières plastiques et ouvrages en ces matières,109.230,107.130,2.100,-105.030,1.020,4.730,0.010,🟡 Potentiel modéré - Équilibré,0


In [17]:
# 8.3 Créer le rapport d'évaluation complet
evaluation_report = {
    'date_evaluation': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
    'model_performance': {
        'regression': {
            'train': metrics_reg['Train'],
            'test': metrics_reg['Test'],
            'cross_validation': {
                'r2_mean': float(cv_scores_reg.mean()),
                'r2_std': float(cv_scores_reg.std()),
                'rmse_mean': float(-cv_scores_rmse.mean())
            }
        },
        'classification': {
            'accuracy': float(accuracy_score(y_test_clf, y_pred_clf)),
            'f1_macro': float(f1_score(y_test_clf, y_pred_clf, average='macro')),
            'precision_macro': float(precision_score(y_test_clf, y_pred_clf, average='macro')),
            'recall_macro': float(recall_score(y_test_clf, y_pred_clf, average='macro')),
            'cross_validation': {
                'accuracy_mean': float(cv_scores_clf.mean()),
                'accuracy_std': float(cv_scores_clf.std())
            }
        }
    },
    'feature_importance': importance_combined.head(15).to_dict('records'),
    'error_analysis': {
        'mean_absolute_error': float(df_test['abs_error'].mean()),
        'median_error': float(df_test['abs_error'].median()),
        'max_error': float(df_test['abs_error'].max()),
        'misclassification_rate': float(len(misclassified) / len(df_test))
    },
    'recommendations_summary': {
        'high_potential_sectors': sector_analysis[
            sector_analysis['classification'].str.contains('Fort')
        ].index.tolist()[:5],
        'priority_sectors': high_import_sectors.head(5).index.tolist() if len(high_import_sectors) > 0 else [],
        'total_substitution_potential_mds_fcfa': float(
            sector_analysis[sector_analysis['ratio_prod_import'] < 1]['imports_fcfa'].sum() -
            sector_analysis[sector_analysis['ratio_prod_import'] < 1]['production_fcfa'].sum()
        )
    }
}

with open(MODELS_PATH / "evaluation_report.json", 'w', encoding='utf-8') as f:
    json.dump(evaluation_report, f, ensure_ascii=False, indent=2, default=str)

print("✅ Rapport d'évaluation sauvegardé: evaluation_report.json")

✅ Rapport d'évaluation sauvegardé: evaluation_report.json


## 9. 📝 Résumé et Conclusions

In [18]:
# Résumé final
print("="*80)
print("📊 RAPPORT FINAL D'ÉVALUATION DES MODÈLES")
print("="*80)

print(f"""
✅ ÉVALUATION TERMINÉE AVEC SUCCÈS!

📈 PERFORMANCE DES MODÈLES
══════════════════════════════════════════════════════════════════════════════

🔹 Modèle de Régression (Score de Substitution)
   ─────────────────────────────────────────────
   • R² (Test):           {metrics_reg['Test']['R²']:.4f}
   • RMSE (Test):         {metrics_reg['Test']['RMSE']:.4f}
   • MAE (Test):          {metrics_reg['Test']['MAE']:.4f}
   • CV R² (5-fold):      {cv_scores_reg.mean():.4f} ± {cv_scores_reg.std()*2:.4f}

🔹 Modèle de Classification (Opportunités)
   ─────────────────────────────────────────
   • Accuracy:            {accuracy_score(y_test_clf, y_pred_clf):.4f}
   • F1-Score (macro):    {f1_score(y_test_clf, y_pred_clf, average='macro'):.4f}
   • Erreurs de classif.: {len(misclassified)}/{len(df_test)} ({len(misclassified)/len(df_test)*100:.1f}%)
   • CV Accuracy:         {cv_scores_clf.mean():.4f} ± {cv_scores_clf.std()*2:.4f}

🎯 TOP 5 FEATURES LES PLUS IMPORTANTES
══════════════════════════════════════════════════════════════════════════════
""")
for i, row in importance_combined.head(5).iterrows():
    print(f"   {i+1}. {row['feature']}: {row['importance_avg']:.4f}")

print(f"""

🏆 SECTEURS À FORT POTENTIEL DE SUBSTITUTION
══════════════════════════════════════════════════════════════════════════════
""")
for i, (sector, row) in enumerate(sector_analysis.head(5).iterrows(), 1):
    print(f"   {i}. {sector[:50]}...")
    print(f"      Score: {row['substitution_score_normalized']:.1f} | {row['classification']}")

print(f"""

📁 FICHIERS GÉNÉRÉS POUR LE DÉPLOIEMENT
══════════════════════════════════════════════════════════════════════════════
   • api_config.json          - Configuration pour l'API
   • recommendations_report.json/csv - Recommandations sectorielles
   • evaluation_report.json   - Rapport d'évaluation complet

🚀 PRÊT POUR LE DÉPLOIEMENT!
   L'API et l'application Streamlit peuvent maintenant utiliser ces modèles
   pour fournir des recommandations en temps réel.
""")
print("="*80)

📊 RAPPORT FINAL D'ÉVALUATION DES MODÈLES

✅ ÉVALUATION TERMINÉE AVEC SUCCÈS!

📈 PERFORMANCE DES MODÈLES
══════════════════════════════════════════════════════════════════════════════

🔹 Modèle de Régression (Score de Substitution)
   ─────────────────────────────────────────────
   • R² (Test):           0.8320
   • RMSE (Test):         3.6252
   • MAE (Test):          1.8506
   • CV R² (5-fold):      0.7046 ± 0.2250

🔹 Modèle de Classification (Opportunités)
   ─────────────────────────────────────────
   • Accuracy:            0.9841
   • F1-Score (macro):    0.9891
   • Erreurs de classif.: 1/63 (1.6%)
   • CV Accuracy:         0.9762 ± 0.0522

🎯 TOP 5 FEATURES LES PLUS IMPORTANTES
══════════════════════════════════════════════════════════════════════════════

   18. prix_unitaire_production: 0.2191
   4. consommation_fcfa: 0.1697
   35. production_fcfa_growth: 0.1668
   7. exports_fcfa: 0.1273
   15. taux_couverture: 0.0870


🏆 SECTEURS À FORT POTENTIEL DE SUBSTITUTION
════════════